In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from seghub import segbox

from scribbles_testing import cellpose_data_handler
from scribbles_testing.image_analysis_helpers import single_img_stats

/home/rschwob/mambaforge/envs/dinov2_pip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_folder = "/mnt/imaging.data/rschwob/cellpose_run07/"
output_folder = "/mnt/imaging.data/rschwob/cellpose_run08/"
pred_seed = 123
df_list = []

In [3]:
for img_num in range(2):
    print(f"===== IMG {img_num} =====\n")
    img_base = str(img_num).zfill(3)
    img_path = img_folder + f"{img_base}_img.png"
    img = np.array(Image.open(img_path))#[:,:,1] # NOTE: If we only want to use 1 channel, we can filter here
    img = cellpose_data_handler.preprocess_img(img)
    gt_path = img_folder + f"{img_base}_ground_truth.png"
    ground_truth = np.array(Image.open(gt_path))
    for bin in (0.01, 0.1, 1):
        print(f"=== BIN {bin} ===\n")
        scribbles_suff = f"all_{cellpose_data_handler.bin_for_file(bin)}_w1_run07"
        scribbles_path = img_folder + f"{img_base}_scribbles_{scribbles_suff}.png"
        scribbles = np.array(Image.open(scribbles_path))
        pred_path = output_folder + f"{img_base}_channels_{scribbles_suff[:-1]+'8'}.png"
        sb = segbox.SegBox()
        sb.set_options(img_as_feature=True)
        sb.set_rf_settings(random_state=pred_seed)
        pred = sb.rf_selfpredict(img, scribbles)
        pred_image = Image.fromarray(pred)
        # pred_image.save(pred_path)

        class_1_pix_gt = np.sum(ground_truth == 1)
        class_2_pix_gt = np.sum(ground_truth == 2)
        max_class_pix_gt = max(class_1_pix_gt, class_2_pix_gt)
        min_class_pix_gt = min(class_1_pix_gt, class_2_pix_gt)
        pix_labelled = np.sum(scribbles>0)
        class_1_pix_labelled = np.sum(scribbles == 1)
        class_2_pix_labelled = np.sum(scribbles == 2)
        max_pix_labelled = max(class_1_pix_labelled, class_2_pix_labelled)
        min_pix_labelled = min(class_1_pix_labelled, class_2_pix_labelled)
        pix_in_img = (scribbles.shape[0] * scribbles.shape[1])
        perc_labelled = pix_labelled / pix_in_img * 100
        acc, mPrec, mRecall, mIoU, mF1 = single_img_stats(pred, ground_truth)

        res = pd.DataFrame({'img_num': img_num,
                'prediction tag': "channels",
                'prediction type': "channels",
                'layers': None,
                'scalings': None,
                'scribbles mode': "all",
                'scribbles bin': bin,
                'suffix': "run08",
                'class_1_pix_gt': class_1_pix_gt,
                'class_2_pix_gt': class_2_pix_gt,
                'min_class_pix_gt': min_class_pix_gt,
                'max_class_pix_gt': max_class_pix_gt,
                'pix_labelled': pix_labelled,
                'class_1_pix_labelled': class_1_pix_labelled,
                'class_2_pix_labelled': class_2_pix_labelled,
                'min_class_pix_labelled': min_pix_labelled,
                'max_class_pix_labelled': max_pix_labelled,
                'pix_in_img': pix_in_img,
                'perc. labelled': perc_labelled,
                'accuracy': acc,
                'mPrecision': mPrec,
                'mRecall': mRecall,
                'mIoU': mIoU,
                'mF1': mF1,
                'image': img_path,
                'ground truth': gt_path,
                'scribbles': scribbles_path,
                'prediction': pred_path}, index=[0])
        df_list.append(res)

===== IMG 0 =====

=== BIN 0.01 ===

=== BIN 0.1 ===

=== BIN 1 ===

===== IMG 1 =====

=== BIN 0.01 ===

=== BIN 0.1 ===

=== BIN 1 ===



In [4]:
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df_path = "/mnt/imaging.data/rschwob/scribbles_DFs/"
df_name = "cellpose_run08.csv"
df.to_csv(df_path+df_name, index=False)